# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
filepaths = []
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    for name in files:
        filepaths.append(os.path.join(root, name))
    for name in dirs:
        print(os.path.join(root, name))
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
#To check the number lines of event_datafile_new.csc file. 
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
## The following code is to create cassandra cluster the connection will be established 127.0.0.1.
from cassandra.cluster import Cluster
try:
    cluster = Cluster()
    session = cluster.connect()
except Exception as e:
    print('Exception while connecting to cluster')
    print(e)

#### Create Keyspace

In [6]:
# To create udacity keyspace with 'SimpleStrategy' and replication factor as 1. 
try:
    session.execute(""" CREATE KEYSPACE IF NOT EXISTS udacity 
                         WITH REPLICATION =
                        {'class':'SimpleStrategy', 'replication_factor': 1 } """)
except Exception as e:
    print('Exception while creating keyspace udacity')
    print(e)

#### Set Keyspace

In [7]:
# Using cassandra cluster session will set keyspace value. 
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Question 1 : Give me the artist, song title and song's length in music app history that was heard during sessionId=338, itemInSession=4
## Answer : The question expects artist, song and the song's length Now, thinking of query which is required to get these data is 
## SELECT artist, song_title, song_length FROM play_history WHERE session_id = <value> AND item_in_session = <value>;
## by keeping this query in mind to design NoSQL data model
##     1. We need artist, song, song's length as output.
##     2. Based on unique session id and itemInSession. 
## As we know the SELECT query, we can move to CREATE table query. We will add NOT EXIST to the CREATE statement to check if 
## the table exists and only create the table if it does not exist. Now we need to select the columns that are going to be in 
## the table and the PRIMARY KEY.Here the PRIMARY KEY is the composit of song and user_id and CLUSTER column is user.
## COLUMN Names : 
##            1. session_id : integer value of session which the song was/is played.
##.           3. item_in_session : integer value of the item the song which are added in session. 
##            4. artist : text value of the artist of the song. 
##.           5. song_title : text value of the song tittle. 
##            6. song_length : floating type value of the song length. 
## PRIMARY KEY : The primary key should uniquely identify each row. For this question we need the results based on session_id and item_in_session. 
##              So, we are keeping session_id and item_in_session as primary key. 

query = "CREATE TABLE IF NOT EXISTS play_history "
query = query + """( session_id int,
                     item_in_session int,
                     artist text,
                     song_title text,
                     song_length float,
                     PRIMARY KEY (session_id, item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [10]:
# Here the Primary Key has two fields: session_id is the partition key, 
# and item_in_session are clustering keys. Partitioning is done by session_id and 
# within that partition, rows are ordered by the item_in_session.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """ INSERT INTO play_history (session_id, 
                                              item_in_session,
                                              artist, 
                                              song_title, 
                                              song_length) """
        query = query + " VALUES (%s, %s, %s, %s ,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]),int(line[3]), line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## The code to Query the data inserted into play_history table. 
resultQuery = session.execute(""" 
                SELECT artist, song_title, song_length 
                FROM play_history 
                WHERE session_id = 338 AND item_in_session = 4;

""")
print(resultQuery._current_rows)

[Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)]


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182
## Answer : The question expects artist, song which should be sorted by itemInSession adn user's first and last name. 
#  Now, thinking of query which is required to get these data is 
## SELECT artist, song, user FROM play_history_user_by_session WHERE user_id = <value> AND session_id = <value>;
## by keeping this query in mind to design NoSQL data model
##     1. We need artist, song, user as output.
##     2. Based on unique user and session id value. 
## As we know the SELECT query, we can move to CREATE table query. We will add NOT EXIST to the CREATE statement to check if 
## the table exists and only create the table if it does not exist. Now we need to select the columns that are going to be in 
## the table and the PRIMARY KEY.Here the PRIMARY KEY is the composit of song and user_id and CLUSTER column is user.
## COLUMN Names : 
##            1. user_id : integer value of user id.
##            2. session_id : integer value of session which the song was/is played.
##.           3. item_in_session : integer value of the item the song which are added in session. 
##            4. artist : text value of the artist of the song. 
##.           5. song : text value of the song tittle. 
##            6. user : text value of concatinated first and last name of the user_id.
## PRIMARY KEY : The primary key should uniquely identify each row. For this question we need the results based on user_id and session_id. So, we are
##               keeping user_id and session_id as primary key. 
## CLUSTER COLUMN : The results are required to be sorted based on item_in_session. So, we are adding item_in_session as a cluster column.
query = "CREATE TABLE IF NOT EXISTS play_history_user_by_session "
query = query + """( user_id int,
                     session_id int,
                     item_in_session int,
                     artist text,
                     song text,
                     user text,
                     PRIMARY KEY ((user_id, session_id), item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)                      

In [13]:
#Insert data into table play_history_user_by_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """ INSERT INTO play_history_user_by_session (
                                                            user_id, 
                                                            session_id, 
                                                            item_in_session,
                                                            artist,
                                                            song,
                                                            user)
        
        """
        query = query + " VALUES (%s, %s, %s, %s ,%s, %s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]),line[0],line[9],line[1]+" "+line[4]))

In [14]:
## Here the primary key is the composite key user_id and session_id. And we have CLUSTERING COLUMN as item_in_session
## The parition is done based on the composite key and stored sorted based on item_in_session.
## Exectue select query for play_history_user_by_session
resultQuery1 = session.execute(""" 
                SELECT artist, song, user 
                FROM play_history_user_by_session 
                WHERE user_id = 10 AND session_id = 182;

""")
print(resultQuery1._current_rows)

[Row(artist='Down To The Bone', song="Keep On Keepin' On", user='Sylvie Cruz'), Row(artist='Three Drives', song='Greece 2000', user='Sylvie Cruz'), Row(artist='Sebastien Tellier', song='Kilometer', user='Sylvie Cruz'), Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user='Sylvie Cruz')]


In [21]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## Answer : The question expects user first and last name. Now, thinking of query which we could achieved is 
## SELECT user FROM play_history_user_by_song WHERE song = <value> by keeping the query in mind to design NoSQL data model
## 1. We need user (first and last name) as output.
## 2. Based on song unique to the user. 
## As we know the SELECT query, we can move to CREATE table query. We will add NOT EXIST to the CREATE statement to check if 
## the table exists and only create the table if it does not exist. Now we need to select the columns that are going to be in 
## the table and the PRIMARY KEY.Here the PRIMARY KEY is the composit of song and user_id and CLUSTER column is user.
## COLUMN Names : We need song which is title of the song, user_id to uniquely identify the user listened to the song and user which is 
## concatination of fisrt_name and last_name. 

query = "CREATE TABLE IF NOT EXISTS play_history_user_by_song"
query = query + """( song text,
                     user_id int,
                     user text,
                     PRIMARY KEY (song,user_id) )"""

try:
    session.execute(query)
except Exception as e:
    print(e)  
                    

In [22]:
#Insert data into table play_history_user_by_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """ INSERT INTO play_history_user_by_song (song, user_id, user)
        
        """
        query = query + " VALUES (%s, %s ,%s)"
        session.execute(query, (line[9],int(line[10]),line[1]+" "+line[4]))

In [23]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## Answer : The question expects user first and last name. Here the primary key is the composite key song and user.
## The parition is done based on the composite key song and user.
resultQuery3 = session.execute(""" 
                SELECT user 
                FROM play_history_user_by_song 
                WHERE song = 'All Hands Against His Own';

""")
print(resultQuery3._current_rows)

[Row(user='Jacqueline Lynch'), Row(user='Tegan Levine'), Row(user='Sara Johnson')]


### Drop the tables before closing out the sessions

In [24]:
## Drop the table before closing out the sessions
dropQuery_play_history = "drop table play_history"
dropQuery_play_history_user_by_session = "drop table play_history_user_by_session"
dropQuery_play_history_user_by_song  = "drop table play_history_user_by_song"
try:
    session.execute(dropQuery_play_history)
    session.execute(dropQuery_play_history_user_by_session)
    session.execute(dropQuery_play_history_user_by_song)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()